In [ ]:
import torch
from torch import nn
import timeit

In [ ]:
class SharedLayers(nn.Module) :

    # First, output the classification results,
    #   then output the regression values of valence and arousal?
    def __init__(self) :

        super(SharedLayers, self).__init__()
        self.shared_layers = nn.Sequential(
            nn.Conv2d(3, 64, 5, 1),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            
            nn.Conv2d(64, 138, 3, 1),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2,2),

            nn.Conv2d(128, 128, 3, 1),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),

            nn.Conv2d(128, 138, 3, 1),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2,2)
        )

        
    
    def forward(self, input_to_shared) :
        output_from_shared = self.shared_layers(input_to_shared)
        return output_from_shared
        
        

In [ ]:
class BranchLayers(nn.Module) :
    
    def __init__(self) :

        super(BranchLayers, self).__init__()
        self.branch_layers = nn.Sequential(
            nn.Conv2d(128, 128, 3, 1),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),

            nn.Conv2d(128, 256, 3, 1),
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2,2),

            nn.Conv2d(256, 256, 3, 1),
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),

            nn.Conv2d(256, 512, 3, 1),
            nn.BatchNorm2d(512),
            nn.ReLU(inplace=True),
            nn.AdaptiveAvgPool2d(1),

            nn.Linear(512, 8),     # 512 size of image array, 8 out_features (the classification?)
            
            # comment the line below out if do not want to get the valence/arousal values
            nn.Linear(8, 2)        # 8 in_features (the classification?), 2 out_features (the regression?) only an experiment
        )
        
    def forward(self, shared_to_branch) :
        output_from_shared = self.branch_layers(shared_to_branch)
        return output_from_shared      

In [ ]:
class Ensemble(nn.Module) :
    def __init__(self, num_branches=1) :

        super(Ensemble, self).__init__()
        self.shared_layers = SharedLayers()
        self.branch_layers = [BranchLayers()] * num_branches

    
    def get_ensemble_size(self) :
        return len(self.branch_layers)


    def add_branch(self) :
        self.branch_layers.append(BranchLayers()) 


    def remove_branch(self) :
        self.branch_layers.pop()


    def forward(self, x) :
        x_branch = self.shared_layers(x)

        y = []
        for branch in self.branch_layers :
            y.append(branch(x_branch))

        return y    
    
    # also have in the training procedure so might not need to use this function
    def to_device(self) :
        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        self.to(device)
        self.shared_layers.to(device)
        
        for branch in self.branch_layers :
            branch.to(device)

In [ ]:
class Training_Procedure :
    
    def __init__(
        self, model,
        epochs, optimizer, loss_func,
        lr = 0.001
    ) :
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

        self.model = model.to(self.device)
        self.epochs = epochs
        self.optimizer = optimizer
        self.loss_func = loss_func
        self.lr = lr
        

    def train(self, train_data) :
        # setting all the gradient to zero
        self.optimizer.zero_grad()

        # training
        start_time = timeit.default_timer()
        train_output = self.model(train_data)
        
        # calculating loss
        loss = self.loss_func(train_output)
        


    def evaluate(self) :
        pass

    